In [1]:
import numpy as np
import pandas as pd

In [24]:
import random

num_user = 600
num_item = 1000
user_dim = random.randint(2, 10)
item_dim = random.randint(2, 10)
is_range = random.randint(3, 6)
is_dict = np.random.random(is_range)
print(f"{user_dim=}, {item_dim=}, {is_range=}")
print(f"{is_dict}")

user_dim=10, item_dim=7, is_range=5
f[0.67198588 0.29576218 0.52831005 0.89829115 0.34665218]


In [29]:
multipliers = np.random.random(4)

In [25]:
u = np.random.random(user_dim)
us = np.random.random(10)
i = np.random.random(item_dim)
is_ = random.randint(0, is_range)
print(f"{u=}, {us=}, {i=}, {is_=}")

u=array([0.28135906, 0.20752037, 0.32728962, 0.74666038, 0.26535119,
       0.06450621, 0.75289941, 0.64379335, 0.82444148, 0.48471891]), us=array([0.68883409, 0.48935575, 0.90730867, 0.51565162, 0.99445842,
       0.46564106, 0.17557296, 0.3682905 , 0.97204727, 0.79454891]), i=array([0.13965554, 0.13579684, 0.80622216, 0.55018796, 0.25431392,
       0.55087856, 0.59439258]), is_=1


In [28]:
# generate users and items
all_users = np.random.random((num_user, user_dim))
all_items = np.random.random((num_item, item_dim))
all_user_side = np.random.random((num_user, 10))
all_item_side = np.random.randint(0, is_range, (num_item,))

In [27]:
import itertools


def f(u: np.array, i: np.array):
    min_size = min(u.size, i.size)
    return np.dot(u[:min_size], i[:min_size]) + u[min_size:].sum() + i[min_size:].sum()


def g(u: np.array, us: np.array):
    result = 0
    for u_element, us_element in itertools.product(u, us):
        result += u_element * us_element
    return result


def gamma(i: np.array, is_: int):
    return is_dict[is_] * i.sum()


def lambda_(us: np.array, i: np.array):
    return us.sum() + i.sum()

9.403156834137247


In [54]:
from sklearn.preprocessing import MinMaxScaler

TOTAL_ROWS = 10000
NOISE_STD = 0.2

data = np.empty((0, 3))
for i in range(TOTAL_ROWS):
    user_id = random.randint(0, num_user - 1)
    item_id = random.randint(0, num_item - 1)
    u, us = all_users[user_id], all_user_side[user_id]
    i, is_ = all_items[user_id], all_item_side[user_id]
    rating = np.dot(
        [f(u, i), g(u, us), gamma(i, is_), lambda_(us, i)], multipliers
    ) + np.random.normal(scale=NOISE_STD)
    data = np.vstack([data, [user_id, item_id, rating]])

RATING_SCALE = (0, 5)
data[:, 2] = (
    MinMaxScaler(RATING_SCALE).fit_transform(data[:, 2].reshape(-1, 1)).flatten()
)

In [58]:
df = pd.DataFrame(data, columns=["user_id", "item_id", "rating"]).astype(
    dtype={"user_id": int, "item_id": int}
)
df.describe()

,user_id,item_id,rating
count,10000.000000,10000.000000,10000.000000
mean,297.130700,497.990800,2.368615
std,173.420571,290.448644,0.836733
min,0.000000,0.000000,0.000000
25%,145.000000,243.000000,1.795520
50%,295.000000,497.000000,2.324304
75%,447.250000,751.000000,2.884365
max,599.000000,999.000000,5.000000


In [69]:
import os

OUTPUT_PATH = "fake_data"
df.head(5000).to_csv(os.path.join(OUTPUT_PATH, "train.csv"), index=False)
df.tail(5000).to_csv(os.path.join(OUTPUT_PATH, "test.csv"), index=False)
pd.DataFrame(all_user_side, columns=[f"V{i}" for i in range(1, 11)]).to_csv(
    os.path.join(OUTPUT_PATH, "user_feats.csv"), index_label="user_id"
)
all_fake_item_side = np.random.randint(0, is_range, (num_item, 4))
all_fake_item_side[:, random.randint(0, 3)] = all_item_side
pd.DataFrame(all_fake_item_side, columns=[f"V{i}" for i in range(1, 5)]).to_csv(
    os.path.join(OUTPUT_PATH, "item_feats.csv"), index_label="item_id"
)